In [1]:
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from medmnist import INFO
import numpy as np
import faiss
import copy
from tqdm import tqdm 

import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [3]:
from base_models import * 

%load_ext autoreload
%autoreload 1

In [4]:
CHANNELS = 1
N_CLASSES = 2
N_EPOCHS = 15

# DATASET_NAME = "PneumoniaMNIST"
# DATASET_NAME = "DermaMNIST" 
# DATASET_ID = "dermamnist"
DATASET_NAME = "BreastMNIST"
DATASET_ID = "breastmnist" 

### Data Loading 

In [5]:
print(f"Step 1: Loading {DATASET_NAME} dataset with resize transform...")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

info = INFO[DATASET_ID]
DataClass = getattr(__import__('medmnist'), info['python_class'])

device = torch.device('cuda:0')

train_dataset = DataClass(split='train', transform=transform, download=True, size=224)
val_dataset   = DataClass(split='val',   transform=transform, download=True, size=224)
test_dataset  = DataClass(split='test',  transform=transform, download=True, size=224)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=1, shuffle=False)

print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")

Step 1: Loading BreastMNIST dataset with resize transform...
Train samples: 546, Validation samples: 78, Test samples: 156


In [6]:
def train_model(model_class, seed, epochs=10):
    torch.manual_seed(seed)
    model = model_class().to(device)
    print(f"\nTraining {model.__class__.__name__} with seed {seed} for {epochs} epochs...") 

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for x, y in train_loader:
            x, y = x.to(device), y.squeeze().long().to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = F.cross_entropy(out, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * x.size(0)
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += x.size(0)

        avg_loss = total_loss / total
        accuracy = correct / total * 100
        print(f"Epoch {epoch}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

    return copy.deepcopy(model.to('cpu')) 

### Define DESImage 

In [7]:
from torch.nn.functional import softmax, cosine_similarity
from collections import Counter


class DESImage: 
    def __init__(self, dsel_dataset, pool): 
        self.dsel_dataset = dsel_dataset
        self.dsel_loader = DataLoader(dsel_dataset, batch_size=64, shuffle=False) 
        self.dino_model = timm.create_model('vit_base_patch16_224.dino', pretrained=True).to(device)
        self.dino_model.eval()  
        self.pool = pool 

        self.suspected_model_votes = [] 
        
        
    def dino_embedder(self, images):
        if images.shape[1] == 1:
            images = images.repeat(1, 3, 1, 1)
        return self.dino_model.forward_features(images)
        

    def fit(self): 
        dsel_embeddings = []
        dsel_labels = []
    
        with torch.no_grad():
            for imgs, labels in self.dsel_loader:
                imgs = imgs.to(device)
                embs = self.dino_embedder(imgs).cpu()  # shape: [B, seq_len, 768]
                dsel_embeddings.append(embs)
                dsel_labels.append(labels)
    
        # Keep as tensor
        dsel_embeddings_tensor = torch.cat(dsel_embeddings).detach().cpu()  # shape: [N, seq_len, 768]
        cls_tensor = dsel_embeddings_tensor[:, 0, :]  # CLS token only, shape: [N, 768]
    
        # Convert to NumPy
        cls_embeddings = np.ascontiguousarray(cls_tensor.numpy(), dtype='float32')
        self.dsel_embeddings = cls_embeddings
        self.dsel_labels = torch.cat(dsel_labels).numpy()
    
        # Sanity check
        print(type(cls_embeddings))  # should be <class 'numpy.ndarray'>
        print(cls_embeddings.dtype)  # float32
        print(cls_embeddings.flags['C_CONTIGUOUS'])  # True
    
        # Build FAISS index
        embedding_dim = cls_embeddings.shape[1]
        self.index = faiss.IndexFlatL2(embedding_dim)
        self.index.add(cls_embeddings)


    def get_output_size(self, model):
        """
        Returns the output size (number of classes) from various model architectures.
        """
        if hasattr(model, 'fc'):
            return model.fc.out_features
        elif hasattr(model, 'classifier'):
            if isinstance(model.classifier, nn.Sequential):
                return model.classifier[-1].out_features
            else:
                return model.classifier.out_features
        elif hasattr(model, 'heads'):  # ViT / DINO from torchvision
            return model.heads.head.out_features
        elif hasattr(model, 'head'):  # ViT/Swin from timm
            return model.head.out_features
        else:
            raise AttributeError("Cannot determine output size of the model.")


    def predict_weighted(self, test_img, k=7, return_logits=False):
        # Step 1: Get DINO embedding
        img_for_dino = test_img.unsqueeze(0).to(device)
        if img_for_dino.shape[1] == 1:
            img_for_dino = img_for_dino.repeat(1, 3, 1, 1)
        with torch.no_grad():
            test_emb = self.dino_model.forward_features(img_for_dino).cpu().numpy().astype('float32')
            test_emb = test_emb[:, 0, :]  # CLS token only
    
        # Step 2: Nearest neighbors in FAISS
        _, neighbors = self.index.search(test_emb, k)
        neighbor_idxs = neighbors[0]
        local_labels = self.dsel_labels[neighbor_idxs]
    
        # Step 3: Competence of classifiers
        competences = []
        for clf in self.pool:
            clf.eval()
            with torch.no_grad():
                roc_imgs = torch.stack([self.dsel_dataset[idx][0] for idx in neighbor_idxs]).to(device)
                outputs = clf(roc_imgs)
                preds = outputs.argmax(dim=1).cpu().numpy()
            correct = (preds == local_labels).sum()
            competence = correct / k
            competences.append(competence)
    
        # Normalize competences
        total_comp = sum(competences)
        if total_comp == 0:
            competences = [1.0 / len(self.pool)] * len(self.pool)
        else:
            competences = [c / total_comp for c in competences]
    
        # Step 4: Weighted sum of classifier outputs
        test_img = test_img.unsqueeze(0).to(device)  # Add batch dimension
        num_classes = self.get_output_size(self.pool[0].model)
        weighted_logits = torch.zeros(num_classes).to(device)
    
        for clf, weight in zip(self.pool, competences):
            clf.eval()
            with torch.no_grad():
                logits = clf(test_img).squeeze(0)
                weighted_logits += weight * logits
    
        if return_logits:
            return weighted_logits  # shape: (num_classes,)
        
        final_pred = weighted_logits.argmax().item()
        return final_pred

        from torch.nn.functional import softmax, cosine_similarity


    def print_attack_statistics(self):
        if not self.suspected_model_votes:
            print("No predictions made yet.")
            return
    
        total = len(self.suspected_model_votes)
        count = Counter(self.suspected_model_votes)
        print("\n Suspected Attacked Model Statistics:")
        for model_idx, votes in count.items():
            percent = 100.0 * votes / total
            print(f" - Model #{model_idx}: {votes}/{total} ({percent:.2f}%)")
            print(f" Suspected attacked model name: {self.pool[model_idx].__class__.__name__}")
    
        
    
    def predict_weighted_robust(self, test_img, k=7, return_logits=False):
        # Step 1: Get DINO embedding of the test image
        img_for_dino = test_img.unsqueeze(0).to(device)
        if img_for_dino.shape[1] == 1:
            img_for_dino = img_for_dino.repeat(1, 3, 1, 1)
        with torch.no_grad():
            test_emb = self.dino_model.forward_features(img_for_dino).cpu().numpy().astype('float32')
            test_emb = test_emb[:, 0, :]  # CLS token only
    
        # Step 2: Get Region of Competence (RoC) from FAISS
        _, neighbors = self.index.search(test_emb, k)
        neighbor_idxs = neighbors[0]
        local_labels = self.dsel_labels[neighbor_idxs]
        
        # Step 3: Get RoC images
        with torch.no_grad():
            roc_imgs = torch.stack([self.dsel_dataset[idx][0] for idx in neighbor_idxs]).to(device)
            # if roc_imgs.shape[1] == 1:
            #     roc_imgs = roc_imgs.repeat(1, 3, 1, 1)
    
        # Step 4: Evaluate classifiers — compute competence and softmax outputs
        competences = []
        soft_outputs = []
    
        for clf in self.pool:
            clf.eval()
            with torch.no_grad():
                # Competence from RoC
                outputs = clf(roc_imgs)
                preds = outputs.argmax(dim=1).cpu().numpy()
                correct = (preds == local_labels).sum()
                competence = correct / k
                competences.append(competence)
    
                # Softmax on test image
                logits = clf(test_img.unsqueeze(0).to(device)).squeeze(0)
                probs = softmax(logits, dim=0)
                soft_outputs.append(probs)
    
        # Step 5: Agreement filtering via cosine similarity to average prediction
        mean_probs = torch.stack(soft_outputs).mean(dim=0)
        mean_probs_normed = mean_probs / mean_probs.norm()
    
        similarities = []
        for probs in soft_outputs:
            probs_normed = probs / probs.norm()
            sim = cosine_similarity(probs_normed.unsqueeze(0), mean_probs_normed.unsqueeze(0), dim=1)
            similarities.append(sim.item())
    
        # Step 6: Combine competence and similarity
        alpha = 2.0  # adjust influence of agreement
        combined_scores = [c * np.exp(alpha * s) for c, s in zip(competences, similarities)]
    
        # Normalize scores to get weights
        total_score = sum(combined_scores)
        if total_score == 0:
            weights = [1.0 / len(self.pool)] * len(self.pool)
        else:
            weights = [s / total_score for s in combined_scores]
    
        # Step 7: Weighted aggregation of classifier predictions
        num_classes = self.get_output_size(self.pool[0].model)
        weighted_logits = torch.zeros(num_classes).to(device)
        for prob, weight in zip(soft_outputs, weights):
            weighted_logits += weight * prob
    
        # Step 8: Print the suspected attacked model
        min_sim_idx = int(np.argmin(similarities))
        self.suspected_model_votes.append(min_sim_idx) 
    
        if return_logits:
            return weighted_logits  # shape: (num_classes,)
        return weighted_logits.argmax().item()


### Benchmark 

In [8]:
from torch.utils.data import TensorDataset, DataLoader
from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from collections import defaultdict


def evaluate_classifiers(pool, test_dataset, n_classes):
    loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    results = []

    for i, model in enumerate(pool):
        model = model.to(device)
        model.eval()
        y_true, y_pred, y_prob = [], [], []

        with torch.no_grad():
            for imgs, labels in loader:
                imgs, labels = imgs.to(device), labels.squeeze().long().to(device)
                outputs = model(imgs)
                preds = outputs.argmax(dim=1)
                probs = torch.softmax(outputs, dim=1)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())
                y_prob.extend(probs.cpu().numpy())

        acc = (np.array(y_true) == np.array(y_pred)).mean() * 100
        f1 = f1_score(y_true, y_pred, average='macro') * 100
        try:
            auc = roc_auc_score(y_true, y_prob, multi_class='ovr') * 100
        except ValueError:
            auc = float('nan')  # Handle single-class test sets

        results.append({
            'classifier': model.__class__.__name__,
            'accuracy': acc,
            'f1': f1,
            'auc': auc,
        })

    return results

In [9]:
import time

total_start = time.time()  # ⏱️ Start timing 

SEEDS = [10, 33, 45, 67, 85, 99, 102, 129, 141, 167] 
# SEEDS = [10, 33]

all_results = defaultdict(list) 

des_metrics = {
    'accuracy': [],
    'f1': [],
    'auc': []
}

attacked_results = defaultdict(list)
des_attacked_metrics = {
    'accuracy': [],
    'f1': [],
    'auc': []
}

des_defense_metrics = {
    'accuracy': [],
    'f1': [],
    'auc': []
}


for seed in SEEDS: 
    print(f"\n--- Seed: {seed} ---")
    trained_resnet18 = train_model(lambda: ResNet18Classifier(N_CLASSES, CHANNELS).to(device), seed, N_EPOCHS) 
    trained_resnet34 = train_model(lambda: ResNet34Classifier(N_CLASSES, CHANNELS).to(device), seed, N_EPOCHS) 
    trained_resnet50 = train_model(lambda: ResNet50Classifier(N_CLASSES, CHANNELS), seed, N_EPOCHS) 
    trained_mobilenetv2 = train_model(lambda: MobileNetV2Classifier(N_CLASSES, CHANNELS).to(device), seed, N_EPOCHS) 
    trained_efficientnet = train_model(lambda: EfficientNetB0Classifier(N_CLASSES, CHANNELS), seed, N_EPOCHS) 
    trained_densenet201 = train_model(lambda: DenseNet201Classifier(N_CLASSES, CHANNELS), seed, N_EPOCHS)  
    trained_shufflenet = train_model(lambda: ShuffleNetClassifier(N_CLASSES, CHANNELS), seed, N_EPOCHS)
    trained_googlenet = train_model(lambda:GoogLeNetClassifier(N_CLASSES, CHANNELS), seed, N_EPOCHS) 

    pool = [
        trained_resnet18, 
        trained_resnet34, 
        trained_resnet50, 
        trained_mobilenetv2,
        trained_efficientnet, 
        trained_densenet201,
        trained_shufflenet,
        trained_googlenet
    ]

    seed_results = evaluate_classifiers(pool, test_dataset, N_CLASSES)

    for res in seed_results:
        all_results[res['classifier']].append(res)


    # Build DES model
    des_model = DESImage(val_dataset, pool)
    des_model.fit()

    # Evaluate on test set
    y_true, y_pred, y_prob = [], [], []

    for imgs, labels in tqdm(test_loader):
        img = imgs[0]
        label = labels.item()

        logits = des_model.predict_weighted(img, return_logits=True)
        probs = torch.softmax(logits, dim=0).cpu().numpy()
        pred = np.argmax(probs)

        y_true.append(label)
        y_pred.append(pred)
        y_prob.append(probs)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.vstack(y_prob)

    # Compute metrics
    acc = accuracy_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred, average='macro') * 100
    try:
        auc = roc_auc_score(y_true, y_prob, multi_class='ovr') * 100
    except:
        auc = float('nan')

    print(f"DES Accuracy: {acc:.2f}%  | F1: {f1:.2f}%  | AUC: {auc:.2f}%")

    des_metrics['accuracy'].append(acc)
    des_metrics['f1'].append(f1)
    des_metrics['auc'].append(auc)

    ### Attack =============================================================================================
    target_model = pool[0]  
    target_model = target_model.to(device).eval() 

    dummy_optimizer = torch.optim.Adam(target_model.parameters())
    loss_fn = nn.CrossEntropyLoss() 

    # Wrap it for ART
    art_classifier = PyTorchClassifier(
        model=target_model,
        loss=loss_fn,
        optimizer=dummy_optimizer,
        input_shape=(1, 224, 28),  
        nb_classes=N_CLASSES,
        clip_values=(0.0, 1.0),
    )

    x_test = []
    y_test = []
    
    for img, label in test_dataset:
        x_test.append(img.numpy())
        y_test.append(label.item())
    
    x_test = np.array(x_test)
    y_test = np.array(y_test)

    pgd_attack = ProjectedGradientDescent(estimator=art_classifier, eps=0.01, eps_step=0.01, max_iter=100)
    x_test_adv = pgd_attack.generate(x=x_test)

    adv_tensor = torch.tensor(x_test_adv).float()
    labels_tensor = torch.tensor(y_test).long()
    adv_dataset = TensorDataset(adv_tensor, labels_tensor)

    adv_test_loader = DataLoader(adv_dataset, batch_size=1, shuffle=False)  

    seed_attack_results = evaluate_classifiers(pool, adv_dataset, N_CLASSES) 
    for res in seed_attack_results:
        attacked_results[res['classifier']].append(res)


    # Evaluate on test set
    y_true, y_pred, y_prob = [], [], []

    for imgs, labels in tqdm(adv_test_loader):
        img = imgs[0]
        label = labels.item()

        logits = des_model.predict_weighted(img, return_logits=True)
        probs = torch.softmax(logits, dim=0).cpu().numpy()
        pred = np.argmax(probs)

        y_true.append(label)
        y_pred.append(pred)
        y_prob.append(probs)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.vstack(y_prob)

    # Compute metrics
    acc = accuracy_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred, average='macro') * 100
    try:
        auc = roc_auc_score(y_true, y_prob, multi_class='ovr') * 100
    except:
        auc = float('nan')

    print(f"DES Accuracy: {acc:.2f}%  | F1: {f1:.2f}%  | AUC: {auc:.2f}%")

    des_attacked_metrics['accuracy'].append(acc)
    des_attacked_metrics['f1'].append(f1)
    des_attacked_metrics['auc'].append(auc)


    # DEFENSE ======================================================
    y_true, y_pred, y_prob = [], [], []

    for imgs, labels in tqdm(adv_test_loader):
        img = imgs[0]
        label = labels.item()

        logits = des_model.predict_weighted_robust(img, return_logits=True)
        probs = torch.softmax(logits, dim=0).cpu().numpy()
        pred = np.argmax(probs)

        y_true.append(label)
        y_pred.append(pred)
        y_prob.append(probs)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_prob = np.vstack(y_prob)

    des_model.print_attack_statistics()

    # Compute metrics
    acc = accuracy_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred, average='macro') * 100
    try:
        auc = roc_auc_score(y_true, y_prob, multi_class='ovr') * 100
    except:
        auc = float('nan')

    print(f"DES Accuracy: {acc:.2f}%  | F1: {f1:.2f}%  | AUC: {auc:.2f}%")

    des_defense_metrics['accuracy'].append(acc)
    des_defense_metrics['f1'].append(f1)
    des_defense_metrics['auc'].append(auc)


total_end = time.time()
print(f"\n⏱️ Total time for all seeds: {(total_end - total_start) / 60:.2f} minutes")


--- Seed: 10 ---

Training ResNet18Classifier with seed 10 for 15 epochs...
Epoch 1/15 - Loss: 0.6939, Accuracy: 68.50%
Epoch 2/15 - Loss: 0.3429, Accuracy: 87.55%
Epoch 3/15 - Loss: 0.2507, Accuracy: 88.10%
Epoch 4/15 - Loss: 0.1852, Accuracy: 92.12%
Epoch 5/15 - Loss: 0.0947, Accuracy: 97.44%
Epoch 6/15 - Loss: 0.0486, Accuracy: 97.99%
Epoch 7/15 - Loss: 0.0319, Accuracy: 98.72%
Epoch 8/15 - Loss: 0.0482, Accuracy: 98.17%
Epoch 9/15 - Loss: 0.0419, Accuracy: 98.35%
Epoch 10/15 - Loss: 0.0194, Accuracy: 99.63%
Epoch 11/15 - Loss: 0.0187, Accuracy: 99.45%
Epoch 12/15 - Loss: 0.0151, Accuracy: 99.82%
Epoch 13/15 - Loss: 0.0133, Accuracy: 99.82%
Epoch 14/15 - Loss: 0.0088, Accuracy: 99.82%
Epoch 15/15 - Loss: 0.0199, Accuracy: 99.82%

Training ResNet34Classifier with seed 10 for 15 epochs...
Epoch 1/15 - Loss: 0.8100, Accuracy: 71.79%
Epoch 2/15 - Loss: 0.3378, Accuracy: 87.91%
Epoch 3/15 - Loss: 0.2304, Accuracy: 90.84%
Epoch 4/15 - Loss: 0.1903, Accuracy: 92.31%
Epoch 5/15 - Loss: 0.1

100%|█████████████████████████████████████████████████████████████████| 156/156 [00:25<00:00,  6.00it/s]


DES Accuracy: 90.38%  | F1: 87.49%  | AUC: nan%


PGD - Batches:   0%|          | 0/5 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████| 156/156 [00:25<00:00,  6.18it/s]


...

PGD - Batches:   0%|          | 0/5 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████| 156/156 [00:24<00:00,  6.36it/s]


DES Accuracy: 82.69%  | F1: 74.34%  | AUC: nan%


100%|█████████████████████████████████████████████████████████████████| 156/156 [00:20<00:00,  7.64it/s]


 Suspected Attacked Model Statistics:
 - Model #0: 141/156 (90.38%)
 Suspected attacked model name: ResNet18Classifier
 - Model #6: 5/156 (3.21%)
 Suspected attacked model name: ShuffleNetClassifier
 - Model #1: 8/156 (5.13%)
 Suspected attacked model name: ResNet34Classifier
 - Model #5: 2/156 (1.28%)
 Suspected attacked model name: DenseNet201Classifier
DES Accuracy: 87.18%  | F1: 82.60%  | AUC: nan%

⏱️ Total time for all seeds: 34.32 minutes


In [12]:
import pandas as pd 

summary = []

for clf_name, metrics_list in all_results.items():
    accs = [m['accuracy'] for m in metrics_list]
    f1s = [m['f1'] for m in metrics_list]
    aucs = [m['auc'] for m in metrics_list]

    summary.append({
        'Classifier': clf_name,
        'Accuracy': f"{np.mean(accs):.2f} ± {np.std(accs):.2f}",
        'F1': f"{np.mean(f1s):.2f} ± {np.std(f1s):.2f}",
        'AUC': f"{np.mean(aucs):.2f} ± {np.std(aucs):.2f}",
    })

df_summary = pd.DataFrame(summary)
print(df_summary)

                 Classifier       Accuracy             F1        AUC
0        ResNet18Classifier  81.41 ± 10.21  75.38 ± 12.17  nan ± nan
1        ResNet34Classifier   80.13 ± 8.91  74.98 ± 10.10  nan ± nan
2        ResNet50Classifier   82.31 ± 5.02   76.59 ± 6.91  nan ± nan
3     MobileNetV2Classifier   86.54 ± 3.07   82.17 ± 3.69  nan ± nan
4  EfficientNetB0Classifier   87.18 ± 2.20   82.75 ± 2.88  nan ± nan
5     DenseNet201Classifier   81.47 ± 8.36   75.96 ± 7.68  nan ± nan
6      ShuffleNetClassifier   84.81 ± 3.95   79.63 ± 5.02  nan ± nan
7       GoogLeNetClassifier   82.69 ± 4.27   77.84 ± 4.40  nan ± nan


In [13]:
import pandas as pd

summary = {
    'Metric': [],
    'Mean ± Std': []
}

for metric in ['accuracy', 'f1', 'auc']:
    values = des_metrics[metric]
    mean = np.nanmean(values)
    std = np.nanstd(values)
    summary['Metric'].append(metric.upper())
    summary['Mean ± Std'].append(f"{mean:.2f} ± {std:.2f}")

df_summary = pd.DataFrame(summary)
print("\n=== Final DES Performance (across seeds) ===")
print(df_summary)


=== Final DES Performance (across seeds) ===
     Metric    Mean ± Std
0  ACCURACY  89.87 ± 1.28
1        F1  86.28 ± 2.19
2       AUC     nan ± nan


# <div style="color:white;display:fill;border-radius:5px;background-color:#f02263;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:110%"><b></b> Adversarial Attack Results</p></div>

In [14]:
import pandas as pd 

summary = []

for clf_name, metrics_list in attacked_results.items():
    accs = [m['accuracy'] for m in metrics_list]
    f1s = [m['f1'] for m in metrics_list]
    aucs = [m['auc'] for m in metrics_list]

    summary.append({
        'Classifier': clf_name,
        'Accuracy': f"{np.mean(accs):.2f} ± {np.std(accs):.2f}",
        'F1': f"{np.mean(f1s):.2f} ± {np.std(f1s):.2f}",
        'AUC': f"{np.mean(aucs):.2f} ± {np.std(aucs):.2f}",
    })

df_summary = pd.DataFrame(summary)
print(df_summary)

                 Classifier       Accuracy             F1        AUC
0        ResNet18Classifier  21.60 ± 10.94   19.82 ± 8.95  nan ± nan
1        ResNet34Classifier   78.78 ± 9.92  73.39 ± 10.85  nan ± nan
2        ResNet50Classifier   81.15 ± 6.95   75.74 ± 7.68  nan ± nan
3     MobileNetV2Classifier   85.71 ± 2.95   80.62 ± 4.08  nan ± nan
4  EfficientNetB0Classifier   86.73 ± 2.13   81.88 ± 3.09  nan ± nan
5     DenseNet201Classifier   80.19 ± 8.70   74.93 ± 7.94  nan ± nan
6      ShuffleNetClassifier   82.63 ± 4.89   77.18 ± 5.09  nan ± nan
7       GoogLeNetClassifier   81.47 ± 5.29   76.38 ± 5.42  nan ± nan


In [15]:
import pandas as pd

summary = {
    'Metric': [],
    'Mean ± Std': []
}

for metric in ['accuracy', 'f1', 'auc']:
    values = des_attacked_metrics[metric]
    mean = np.nanmean(values)
    std = np.nanstd(values)
    summary['Metric'].append(metric.upper())
    summary['Mean ± Std'].append(f"{mean:.2f} ± {std:.2f}")

df_summary = pd.DataFrame(summary)
print("\n=== Final Attacked DES Performance (across seeds) ===")
print(df_summary)


=== Final Attacked DES Performance (across seeds) ===
     Metric    Mean ± Std
0  ACCURACY  84.17 ± 3.65
1        F1  78.07 ± 6.76
2       AUC     nan ± nan


In [16]:


import pandas as pd

summary = {
    'Metric': [],
    'Mean ± Std': []
}

for metric in ['accuracy', 'f1', 'auc']:
    values = des_defense_metrics[metric]
    mean = np.nanmean(values)
    std = np.nanstd(values)
    summary['Metric'].append(metric.upper())
    summary['Mean ± Std'].append(f"{mean:.2f} ± {std:.2f}")

df_summary = pd.DataFrame(summary)
print("\n=== Final Attacked DES Performance (across seeds) ===")
print(df_summary)


=== Final Attacked DES Performance (across seeds) ===
     Metric    Mean ± Std
0  ACCURACY  88.40 ± 2.54
1        F1  84.29 ± 4.26
2       AUC     nan ± nan
